In [14]:
from google.colab import drive
drive.mount('/content/drive')
% cd /content/drive/MyDrive/Dissertation/ad945-diss-project/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Dissertation/ad945-diss-project


In [2]:
# ! git clone https://github.com/DasAnish/ad945-diss-project.git

Cloning into 'ad945-diss-project'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 175 (delta 18), reused 34 (delta 11), pack-reused 121
Receiving objects: 100% (175/175), 71.31 MiB | 4.42 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [20]:
! git fetch
! git pull

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 14 (delta 5), reused 14 (delta 5), pack-reused 0
Unpacking objects: 100% (14/14), done.
From https://github.com/DasAnish/ad945-diss-project
   1a2f15c..2f4258e  main       -> origin/main
Updating 1a2f15c..2f4258e
Fast-forward
 .../figs/fig4frompaper-trainingtips.PNG                | Bin 0 -> 86363 bytes
 .../figs/length-distribution-across-languages.png      | Bin 0 -> 46069 bytes
 dissertation-tex-files/figs/length-vs-vocab-size.png   | Bin 0 -> 63389 bytes
 dissertation-tex-files/figs/multihead-attention.PNG    | Bin 0 -> 126958 bytes
 dissertation-tex-files/figs/self-attentoin-copied.PNG  | Bin 0 -> 27852 bytes
 .../figs/transformer-architecture.PNG                  | Bin 0 -> 114500 bytes
 src/transformer_layers.py                              |   4 ++--
 7 files changed, 2 insertions(+), 2 deletions(-)
 create mode 100644 dissertation-te

In [3]:
! pip install opennmt-py==1.2.0
! pip install sacrebleu
! pip install sentencepiece
! pip install langdetect
! pip install py-rouge

     |████████████████████████████████| 204kB 7.1MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 
     |████████████████████████████████| 2.6MB 11.6MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=4be5ed1a4a1b79e49d58b859fb56ae1f3b9c40781c110e1c2c3123c1e62acb5f
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 1.2MB 9.3MB/s 
     |████████████████████████████████| 983kB 8.5MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.8-cp37-none-any.whl size=993193 sha256=cb09d23aa6bea1dab2091ee191b60b35e1

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [25]:
from src.transformer_layers import Transformer
from src.utils import *
from src.save_load import *
from src.text_preprocessing import create_fields
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import sqrt
Tensor = torch.Tensor
from torch.autograd import Variable
from typing import Optional
import datetime
import time
import numpy as np
import pickle
import sentencepiece as spm
from tqdm import tqdm
from tqdm.notebook import tnrange
import sacrebleu
from src.opt import Opt
from eval.evaluate import *

In [23]:
opt = Opt()

In [28]:
baseline = get_baseline_scores(opt)

In [ ]:
def print_rouge(rouge_score, space=4):
    for r in rouge_score:
        print(f'{" "*space}{r}: ', end='')
        temp_dict = rouge_score[r]
        # for i in temp_dict:
        #     temp_dict[i] = '%.3f'%temp_dict[i]

        # print(temp_dict)
        print('%.3f'%temp_dict['f'])

def print_latex_table(rouge_score, temp):
    temp.append(f"{rouge_score['rouge-1']['f']:.3f}")
    temp.append(f"{rouge_score['rouge-2']['f']:.3f}")
    temp.append(f"{rouge_score['rouge-l']['f']:.3f}")
    temp.append(f"{rouge_score['rouge-w']['f']:.3f}")
    for r in rouge_score:
        v = '%.3f'% rouge_score[r]['f']


def pretty_print(opt):
    scores = get_scores(opt)
    # print(scores['bleu'])
    bleu: sacrebleu.BLEUScore = scores['bleu']
    print(bleu)
    opt.temp.append(f'{bleu.score:.2f}')
    for t in scores:
        if t == 'bleu': continue
        print_latex_table(scores[t], opt.temp)
        print(t.upper())
        print_rouge(scores[t])
    opt.table.append(' & '.join(opt.temp))

src_langs = ['de', 'es', 'fr']
model_nums = {'de':[9, 21, 42, 81, 120, 162],
              'es':[10, 20, 45, 135, 165, 225],
              'fr':[6, 12, 18, 33, 48, 72]}
epoch_size = {'de': 41.3, 'es':35.4, 'fr':18.2/2}
model_fracs = {i:['%.2f'%(j/epoch_size[i]) for j in model_nums[i]] for i in model_nums}
model_nums = {lang: [i*1000 for i in model_nums[lang]] for lang in model_nums}
opt.proper_method = False


for lang in model_nums:
    opt.table = []
    # print(f'{"="*50}{lang}{"="*50}')
    for i, model_num in enumerate(model_nums[lang]):
        opt.temp = [f'{model_num//2000 if lang != "fr" else model_num//1000},000 ({model_fracs[lang][i]})']
        # print(, end='')
        opt.src_lang = lang
        opt.model_num = model_num
        pretty_print(opt)
        print(opt.temp)
    print(' \\\\\n'.join(opt.table) + "\\\\")